In [178]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go


In [179]:
file_dir = 'D:/Astro/StarCatalogues'
file_name = 'Imm Deep Sky Compendium - 2024 - rev5a.xlsm'
file_path = os.path.join(file_dir, file_name)


In [180]:
data = pd.read_excel(file_path, sheet_name="Main", skiprows=6)


c:\Users\Nina\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed



In [181]:
# Fix up the column headers

# Extract column names for columns 0 to 33, and from row 0 for columns 34 onward
header = list(data.columns[:34]) + data.iloc[0, 34:].tolist()
substitutions = {
    'Object Name & Image': 'Object',
    'Unnamed: 1': 'Simbad',
    'Unnamed: 2': 'Aladin',
    'Right Ascension': 'RA_hms',
    'Unnamed: 12': 'RA_deg',
    'Declination': 'Dec_dms',
    'Unnamed: 14': 'Dec_deg',
    'Const.': 'Constellation',
    'Nick.': 'Name',
    'Alt. ID': 'Name_Alt',
    'Surf.': 'Brightness',
    'Inclin.': 'Inclination',
    17: 'Next17',
    19: 'Next19',
    21: 'Next21',
    23: 'Next23',
    'Unnamed': 'Rename',
}
header[-6]='ObjAz'
header[-5]='Key'
header[-4]='FOV'
header = [substitutions.get(col, col) for col in header]
data.columns = header


In [182]:
# Drop junk columns and first 2 rows (header data)
columns_to_drop = ['Simbad', 'Aladin', 'My', 'My.1', 'My.2', 'Next17', 'Next19', 'Next21', 'Next23', 'Time', 'Date', 'Alt.', 'Separ.', 'Index', 'ObjAz', 'Transit Time', 'Add. Filters', 'Sort to ']
data = data.drop(columns=columns_to_drop)
data = data.iloc[2:].reset_index(drop=True)  


In [183]:
data['RA_deg'] = pd.to_numeric(data['RA_deg'], errors='coerce')
data['Dec_deg'] = pd.to_numeric(data['Dec_deg'], errors='coerce')
data['Size'] = pd.to_numeric(data['Size'], errors='coerce')

In [184]:
data.describe()

,Size,RA_deg,Dec_deg
count,3145.000000,3145.000000,3145.000000
mean,18.470626,186.326545,12.175395
std,69.359606,103.263877,34.070522
min,0.050000,0.079167,-80.866667
25%,2.000000,92.350000,-13.170556
50%,4.500000,188.916667,12.328611
75%,13.000000,275.591667,39.405833
max,2400.000000,359.975000,88.172778


In [185]:
data

,Object,Type,Sub,Class,Size,Distance,Diameter,Rating,Notes,RA_hms,...,Sh2,SNR,vdB,HT,SD,OB,SP,FG,Key,FOV
0,Sh2-139,Neb,Em,HII,11.0,9500,30,1,Faint HII region with small brighter arc,223400,...,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sh2-139,0.366667
1,NGC 7309,Gal,Spiral,Sc,1.8,190,100,2,Unusual arm structure - 3 primary arms and 3 s...,223420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NGC 7309,0.2
2,V0375 Lac,Stars,YSO,NaN,3.0,2000,1.8,2,YSO that is the source of the tiny intricate w...,223441,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V0375 Lac,0.2
3,Abell 80,Neb,PN,EHY/q,2.2,6000,3.8,2,HII rim with faint OIII inner region. Shape is...,223446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN A66 80,0.2
4,Barnard 370,Neb,Dark,0,45.0,u,u,0,Very faint - Region of narrow dark lanes,223450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Barnard 370,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,NGC 7293,Neb,PN,BRY/r,12.0,650,2.5,5,"Showcase Helix nebula, one of the closest PN t...",222938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NGC 7293,0.4
3141,vdB 154,Neb,Refl,NaN,10.0,2360,7,3,Blue reflection nebula just north of odd torna...,223111,...,NaN,NaN,154,NaN,NaN,NaN,NaN,NaN,vdB 154,0.333333
3142,NGC 7303,Gal,Spiral,SBbc/z,1.7,150,75,2,"Barred offset core, disturbed disk, dust lanes...",223133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NGC 7303,0.2
3143,UGC 12071,Gal,Spiral,S,1.1,500,175,2,Distant spiral somewhat disturbed with a possi...,223225,...,NaN,NaN,NaN,NaN,NaN,NaN,98,NaN,UGC 12071,0.2


In [186]:
# Clean up the type column
data['Type'] = data['Type'].replace({'Neb ':'Neb'})
type_lookup = {
    'Neb': 'Nebula',
    'Gal': 'Galaxy',
    'Stars': 'Stellar'
}
unique_abbr = set(data['Type'].unique())
valid_abbr = set(type_lookup.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")




All abbreviations are valid and mapped.


In [187]:
# Clean up the constellation column
data['Constellation'] = data['Constellation'].replace({'Apu':'Aps', 'Lmi': 'LMi', 'Uma': 'UMa', 'Crux': 'Cru', 'Cent': 'Cen', 'Pis': 'Pic'})
constellation_lookup = {
    'And': 'Andromeda', 'Ant': 'Antlia', 'Aps': 'Apus', 'Aql': 'Aquila', 'Aqr': 'Aquarius',
    'Ara': 'Ara', 'Ari': 'Aries', 'Aur': 'Auriga', 'Boo': 'Boötes', 'CMa': 'Canis Major',
    'CMi': 'Canis Minor', 'CVn': 'Canes Venatici', 'Cam': 'Camelopardalis', 'Cap': 'Capricornus',
    'Car': 'Carina', 'Cas': 'Cassiopeia', 'Cen': 'Centaurus', 'Cep': 'Cepheus', 'Cet': 'Cetus',
    'Cha': 'Chamaeleon', 'Cir': 'Circinus', 'Cnc': 'Cancer', 'Col': 'Columba', 'Com': 'Coma Berenices',
    'CrA': 'Corona Australis', 'CrB': 'Corona Borealis', 'Crt': 'Crater', 'Cru': 'Crux',
    'Crv': 'Corvus', 'Cyg': 'Cygnus', 'Del': 'Delphinus', 'Dor': 'Dorado', 'Dra': 'Draco',
    'Eri': 'Eridanus', 'For': 'Fornax', 'Gem': 'Gemini', 'Gru': 'Grus', 'Her': 'Hercules',
    'Hor': 'Horologium', 'Hya': 'Hydra', 'LMi': 'Leo Minor', 'Lac': 'Lacerta', 'Leo': 'Leo',
    'Lep': 'Lepus', 'Lib': 'Libra', 'Lup': 'Lupus', 'Lyn': 'Lynx', 'Lyr': 'Lyra',
    'Men': 'Mensa', 'Mic': 'Microscopium', 'Mon': 'Monoceros', 'Mus': 'Musca', 'Nor': 'Norma',
    'Oct': 'Octans', 'Oph': 'Ophiuchus', 'Ori': 'Orion', 'Pav': 'Pavo', 'Peg': 'Pegasus',
    'Per': 'Perseus', 'Pic': 'Pictor', 'PsA': 'Piscis Austrinus', 'Psc': 'Pisces', 'Pup': 'Puppis',
    'Pyx': 'Pyxis', 'Ret': 'Reticulum', 'Scl': 'Sculptor', 'Sco': 'Scorpius', 'Sct': 'Scutum',
    'Ser': 'Serpens', 'Sex': 'Sextans', 'Sge': 'Sagitta', 'Sgr': 'Sagittarius', 'Tau': 'Taurus',
    'Tel': 'Telescopium', 'TrA': 'Triangulum Australe', 'Tri': 'Triangulum', 'Tuc': 'Tucana',
    'UMa': 'Ursa Major', 'UMi': 'Ursa Minor', 'Vel': 'Vela', 'Vir': 'Virgo', 'Vol': 'Volans',
    'Vul': 'Vulpecula'
}
unique_abbr = set(data['Constellation'].unique())
valid_abbr = set(constellation_lookup.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")
data['Constellation'].unique()

grouped = data.groupby(['Constellation', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Constellation', y='Count', color='Type', title='Object Constellation by Type', 
             category_orders={'Constellation': list(constellation_lookup.keys())} ) 
fig.show()



All abbreviations are valid and mapped.


In [188]:
# Clean up the type column
data['Sub'] = data['Sub'].replace({'Group ':'Group', 'Pair ': 'Pair', 'Spiral ': 'Spiral', 'Mol CLd':'Mol Cld', 'Stars':'Star' })
sub_lookup = {
    # Multiple Galaxies
    'Chain': 'Chained Galaxies',
    'Cluster': 'Clustered Galaxies',
    'Group': 'Grouped Galaxies',
    'Merger': 'Merging Galaxies',
    'Pair': 'Pair of Galaxies',
    'Trio': 'Trio of Galaxies',

    # Individual Galaxy
    'BCD': 'Blue Compact Dwarf Galaxy',
    'Coll': 'Collisional Ring Galaxy',
    'Dwarf': 'Dwarf Galaxy',
    'Ellip': 'Elliptical Galaxy',
    'Floc': 'Flocculent Galaxy',
    'Lent': 'Lenticular Galaxy',
    'Mag': 'Magellanic Galaxy',
    'Polar': 'Polar Galaxy',
    'Spiral': 'Spiral Galaxy',

    # Nebula
    'Dark': 'Dark Nebula',
    'Em': 'Emission Nebula',
    'Mol Cld': 'Molecular Cloud Nebula',
    'PN': 'Planetary Nebula',
    'PPN': 'Protoplanetary Nebula',
    'Refl': 'Reflection Nebula',
    'SNR': 'Supernova Remnant Nebula',

    # Stellar associations
    'GC': 'Globular Cluster',
    'HH': 'Herbig-Haro Object',
    'Nova': 'Nova Object',
    'OC': 'Open Cluster',
    'Star': 'Star',
    'Star Cld': 'Star Cloud',
    'YSO': 'Young Stellar Object'
}

unique_abbr = set(data['Sub'].unique())
valid_abbr = set(sub_lookup.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")


# data['SubFull'] = data['Sub'].map(sub_lookup)


grouped = data.groupby(['Sub', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Sub', y='Count', color='Type', title='Object Subtype by Type') 
fig.show()



All abbreviations are valid and mapped.


In [189]:
bins = [    0,      0.5,      1,      2,      5,       10,       30,        100,      3000]
labels = ['<0.5′', '0.5–1′', '1–2′', '2–5′', '5–10′', '10–30′', '30–100′', '100′+']
data['SizeBin'] = pd.cut(data['Size'], bins=bins, labels=labels, include_lowest=True)

# Step 2: Group by SizeBin and Type
grouped = data.groupby(['SizeBin', 'Type'], observed=True).size().reset_index(name='Count')

# Step 3: Plot with color by Type
fig = px.bar(grouped, x='SizeBin', y='Count', color='Type',
             title='Object Size Distribution by Type (Arcmin)',
             category_orders={'SizeBin': labels})  # ensures correct bin order

fig.show()



In [190]:
rating_lookup = {
    5: 'Showcase object in top 2 percent',
    4: 'Excellent object in top 10 percent',
    3: 'Good object in top 25 percent',
    2: 'Typical object seen in most catalogs',
    1: 'Challenging object for advanced imagers',
    0: 'Low priority object not recommended'
}

grouped = data.groupby(['Rating', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Rating', y='Count', color='Type', title='Object Rating by Type' ) 
fig.show()


In [196]:
data['RA_bin'] = (data['RA_deg'] / 5).round() * 5
data['Dec_bin'] = (data['Dec_deg'] / 5).round() * 5
bin_count = data.drop_duplicates(subset=['RA_bin', 'Dec_bin']).shape[0]
bin_count

1202

In [ ]:
import numpy as np
from datetime import datetime, timezone

a='░▁▂▃▄▅▆▇█▔░▁_▂▃▄▅▆▇▔'

def categorize_alt(alt):
    if alt < 0:
        return 'Below Horizon'  # Not visible
    elif alt < 12:
        return 'Near Horizon'   # Rising or setting, poor visibility
    elif alt < 30:
        return 'Low Altitude'   # Often affected by atmospheric distortion
    elif alt < 60:
        return 'Mid Altitude'   # Good visibility, moderate elevation
    elif alt < 82:
        return 'High Altitude'  # Excellent visibility, optimal for imaging
    else:
        return 'Near Zenith'    # Peak elevation, polaris cannot reach


def approx_altaz(ra_deg, dec_deg, observer_lat, observer_lon, time_utc):
    # Convert to radians
    ra = np.radians(ra_deg)
    dec = np.radians(dec_deg)
    lat = np.radians(observer_lat)

    # Julian Date
    jd = (time_utc - datetime(2000, 1, 1, tzinfo=timezone.utc)).total_seconds() / 86400.0 + 2451545.0

    # Local Sidereal Time (LST) in degrees
    lst_deg = (100.46 + 0.985647 * (jd - 2451545.0) + observer_lon + np.degrees(ra)) % 360
    ha_rad = np.radians(lst_deg - ra_deg)  # Hour angle in radians

    # Altitude
    alt_rad = np.arcsin(np.sin(lat) * np.sin(dec) + np.cos(lat) * np.cos(dec) * np.cos(ha_rad))
    alt_deg = np.degrees(alt_rad)

    # Azimuth
    cz = (np.sin(dec) - np.sin(lat) * np.sin(alt_rad)) / (np.cos(lat) * np.cos(alt_rad))
    cz = np.clip(cz, -1, 1)  # Avoid domain errors
    az_rad = np.arccos(cz)
    az_deg = np.degrees(az_rad)
    az_deg = np.where(np.sin(ha_rad) < 0, az_deg, 360 - az_deg)

    return az_deg, alt_deg
